In [ ]:
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import multiprocessing
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
starlink_data = np.load("starlink_data.npy")
print(starlink_data.shape)

In [ ]:
def ts_kmeans_elbow(data: np.ndarray, num_cluster: int):
    '''
    Calculates the sum of squares for different number of clusters using times series K-means
    This allows to select an optimal number of cluster
    '''
    sum_squares = np.zeros(num_cluster-1)
    for i in range(2, num_cluster+1):
        ts_kmeans = TimeSeriesKMeans(n_clusters=num_cluster, metric="dtw", random_state=0, n_jobs=-1).fit(data)
        print(ts_kmeans.inertia_)
        sum_squares[i-2] = ts_kmeans.inertia_
    
    plt.figure(figsize=(12, 7.5))
    plt.plot(np.arange(2, num_cluster+1), sum_squares)
    plt.ylabel('Sum of Squares', fontsize = 14)
    plt.xlabel('Number of clusters', fontsize = 14)
    plt.show()

In [ ]:
# size_nan = np.isnan(starlink_data[:,:,8:9]).sum(axis=1)
# ind_many_nan = np.argwhere(size_nan>2000)

# problem_satellite = starlink_data[ind_many_nan[0][0],:,:]
# problem_satellite = np.nan_to_num(problem_satellite, nan = 50000)
# t = problem_satellite[:,0]
# h = problem_satellite[:,7:10]


# plt.figure(figsize=(12, 7.5))
# plt.plot(size_nan)
# plt.ylabel('Number of NaN', fontsize = 14)
# plt.xlabel('Satellite', fontsize = 14)

# plt.show()

# fig, axs =plt.subplots(3,1, sharex= True)
# axs[0].plot(t,h[:,0])
# axs[0].set_ylabel('h_x')
# axs[1].plot(t,h[:,1])
# axs[1].set_ylabel('h_y')
# axs[2].plot(t,h[:,2])
# axs[2].set_ylabel('h_z')
# axs[2].set_xlabel('Time in J2000')
# axs[0].grid(True)
# axs[1].grid(True)
# axs[2].grid(True)
# fig.subplots_adjust(hspace=0)

In [ ]:
time = starlink_data[:, :, 0]
angular_momentum_x = starlink_data[:, :200, 7]
angular_momentum_y = starlink_data[:, :200, 8]
angular_momentum_z = starlink_data[:, :200, 9]

normed_angular_momentum_x = TimeSeriesScalerMeanVariance().fit_transform(angular_momentum_x)
normed_angular_momentum_y = TimeSeriesScalerMeanVariance().fit_transform(angular_momentum_y)
normed_angular_momentum_z = TimeSeriesScalerMeanVariance().fit_transform(angular_momentum_z)

print(time.shape)
print(normed_angular_momentum_x.shape)

In [ ]:
ts_kmeans_elbow(normed_angular_momentum_x, num_cluster=10)

# ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="dtw", random_state=0, n_jobs=-1)
# ts_kmeans.fit(normed_angular_momentum_x)
# print(ts_kmeans.inertia_)
